In [1]:
# Basic
from collections import Counter
import datetime
import math
import numpy as np
import os
import pprint

# yaml specific
import yaml

# Data handling
from fuzzywuzzy import fuzz, process
import pandas as pd
from tqdm import tqdm

# my library
from db_utils import update_player, add_player 
from core import runs
from insights import total_runs

C:\Users\aniru\miniconda3\envs\torch\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [2]:
# Config variables
raw_data_path = "raw_data"
clean_data_path = "clean_data"
tournament_name = "IPL"

In [3]:
df_tournament = pd.read_csv(os.path.join(clean_data_path, "tournament.csv"))
df_tournament = df_tournament.loc[:, ~df_tournament.columns.str.contains('^Unnamed')]
tournament_id_map = dict(zip(df_tournament.tournament_name, df_tournament.tournament_id))

In [4]:
df_venue = pd.read_csv(os.path.join(clean_data_path, "venue.csv"))
df_venue = df_venue.loc[:, ~df_venue.columns.str.contains('^Unnamed')]
venue_id_map = dict(zip(df_venue.venue_name, df_venue.venue_id))

# Mapping the duplicates as well to its correct venue ids
venue_id_map["Punjab Cricket Association IS Bindra Stadium, Mohali"] = venue_id_map["Punjab Cricket Association Stadium, Mohali"]
venue_id_map["M.Chinnaswamy Stadium"] = venue_id_map["M Chinnaswamy Stadium"]

In [5]:
df_team = pd.read_csv(os.path.join(clean_data_path, "team.csv"))
df_team = df_team.loc[:, ~df_team.columns.str.contains('^Unnamed')]
team_id_map = dict(zip(df_team.team_name, df_team.team_id))

In [6]:
df_player = pd.read_csv(os.path.join(clean_data_path, "player.csv"))
df_player = df_player.loc[:, ~df_player.columns.str.contains('^Unnamed')]
player_dispname_id_map = dict(zip(df_player.player_display_name, df_player.player_id))
player_name_id_map = dict(zip(df_player.player_name, df_player.player_id))

In [7]:
df_match = pd.read_csv(os.path.join(clean_data_path, "match.csv"))
df_match = df_match.loc[:, ~df_match.columns.str.contains('^Unnamed')]

df_ball = pd.read_csv(os.path.join(clean_data_path, "ball.csv"))
df_ball = df_ball.loc[:, ~df_ball.columns.str.contains('^Unnamed')]

C:\Users\aniru\miniconda3\envs\torch\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
player_name = "Virat Kohli"
player_id = player_name_id_map[player_name]
venue_name = "ALL"
match_format = "T20"
tournaments = []
years_range = [str(year) for year in range(2000, 2022)]
overs_range = [0, 20]
top_n=20

In [9]:
#total_runs(player_name, top_n=top_n, match_format=match_format, tournaments=tournaments, venue_name=venue_name, overs_range=overs_range, years_range=years_range)

TypeError: can only concatenate str (not "int") to str

In [11]:
def dismissals(player, tournaments=None, venue=None, years=None, overs_range=None, against_spin=None, against_pace=None, against_bowler=None):
    """
        Total dismissals of this player given the conditions
        Args:
            player - (int) id of target player
            tournaments - (list of ints) list of tournament ids
            venue - (int) id of venue. 
            years - (list) list of years you want to consider
            overs_range - (list) 2 member list denoting [start_over, end_over]
            against_spin - (boolean) mark it true if you want data only specific to spin. dont mark this if you supply 'against_bowler'
            against_pace - (boolean) mark it true if you want data only specific to pace. dont mark this if you supply 'against_bowler'
            against_bowler - (int) id of specific bowler to find data against
    """
    
    # Grabbing all balls faced by this player
    required_balls = df_ball[(df_ball["batsman"] == player) | (df_ball["non_striker"] == player)]
    
    # Grabbing all matches to map ids
    required_matches = df_match
    
    # Run the required_matches dataframe through each match filter
    
    if tournaments is not None:
        required_matches = required_matches[required_matches['tournament_id'].isin(tournaments)]
        
    if venue is not None:
        required_matches = required_matches[required_matches['venue_id'] == venue]
        
    if years is not None:
        required_matches = required_matches[required_matches['match_date'].str.contains('|'.join(years))]
    
    match_ids_to_consider = np.array(required_matches['match_id'])
    required_balls = required_balls[required_balls['match_id'].isin(match_ids_to_consider)]
        
    if overs_range is not None:
        required_balls = required_balls[(required_balls['ball_number'] >= overs_range[0]) & (required_balls['ball_number'] <= overs_range[1])]
    
    if against_bowler is not None:
        required_balls = required_balls[required_balls['bowler'] == against_bowler]

    # TODO
    if against_spin is not None:
        only_spin = True
    
    # TODO
    if against_pace is not None:
        only_pace = True
    
    required_balls = required_balls[required_balls['player_dismissed'] == player]
    print(required_balls)
    
    num_dismissals = len(required_balls)
        
    return num_dismissals

In [12]:
def dismissals(player, tournaments=None, venue=None, years=None, overs_range=None, against_spin=None, against_pace=None, against_bowler=None):
    """
        Total dismissals of this player given the conditions
        Args:
            player - (int) id of target player
            tournaments - (list of ints) list of tournament ids
            venue - (int) id of venue. 
            years - (list) list of years you want to consider
            overs_range - (list) 2 member list denoting [start_over, end_over]
            against_spin - (boolean) mark it true if you want data only specific to spin. dont mark this if you supply 'against_bowler'
            against_pace - (boolean) mark it true if you want data only specific to pace. dont mark this if you supply 'against_bowler'
            against_bowler - (int) id of specific bowler to find data against
    """
    
    # Grabbing all balls faced by this player
    required_balls = df_ball[(df_ball["batsman"] == player) | (df_ball["non_striker"] == player)]
    
    # Grabbing all matches to map ids
    required_matches = df_match
    
    # Run the required_matches dataframe through each match filter
    
    if tournaments is not None:
        required_matches = required_matches[required_matches['tournament_id'].isin(tournaments)]
        
    if venue is not None:
        required_matches = required_matches[required_matches['venue_id'] == venue]
        
    if years is not None:
        required_matches = required_matches[required_matches['match_date'].str.contains('|'.join(years))]
    
    match_ids_to_consider = np.array(required_matches['match_id'])
    required_balls = required_balls[required_balls['match_id'].isin(match_ids_to_consider)]
        
    if overs_range is not None:
        required_balls = required_balls[(required_balls['ball_number'] >= overs_range[0]) & (required_balls['ball_number'] <= overs_range[1])]
    
    if against_bowler is not None:
        required_balls = required_balls[required_balls['bowler'] == against_bowler]

    # TODO
    if against_spin is not None:
        only_spin = True
    
    # TODO
    if against_pace is not None:
        only_pace = True
    
    required_balls = required_balls[required_balls['player_dismissed'] == player]
    
    num_dismissals = len(required_balls)
        
    return num_dismissals

In [13]:
dismissals(player=player_id)

154